In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
def load_data():
    """
    加载原始数据
    """
    print("正在加载数据...")
    # 加载训练集
    train_data = pd.read_csv('../data/used_car_train_20200313.csv', sep=' ')
    # 加载测试集
    test_data = pd.read_csv('../data/used_car_testB_20200421.csv', sep=' ')
    
    print(f"训练集形状: {train_data.shape}")
    print(f"测试集形状: {test_data.shape}")
    
    return train_data, test_data

def preprocess_data(train_data, test_data):
    """
    数据预处理
    """
    print("\n开始数据预处理...")
    
    # 合并训练集和测试集进行特征工程
    train_data['source'] = 'train'
    test_data['source'] = 'test'
    data = pd.concat([train_data, test_data], ignore_index=True)
    
    # 保存SaleID
    train_ids = train_data['SaleID']
    test_ids = test_data['SaleID']
    
    # 从训练集获取y值
    y = train_data['price']
    
    return data, y, train_ids, test_ids

In [4]:
train_data, test_data = load_data()

data, y, train_ids, test_ids = preprocess_data(train_data, test_data)

正在加载数据...
训练集形状: (150000, 31)
测试集形状: (50000, 30)

开始数据预处理...


In [5]:
def create_time_features(data):
    """
    创建时间特征
    """
    print("创建时间特征...")
    
    # 转换日期格式
    data['regDate'] = pd.to_datetime(data['regDate'], format='%Y%m%d', errors='coerce')
    data['creatDate'] = pd.to_datetime(data['creatDate'], format='%Y%m%d', errors='coerce')
    
    # 处理无效日期
    data.loc[data['regDate'].isnull(), 'regDate'] = pd.to_datetime('20160101', format='%Y%m%d')
    data.loc[data['creatDate'].isnull(), 'creatDate'] = pd.to_datetime('20160101', format='%Y%m%d')
    
    # 车辆年龄（天数）
    data['vehicle_age_days'] = (data['creatDate'] - data['regDate']).dt.days
    
    # 修复异常值
    data.loc[data['vehicle_age_days'] < 0, 'vehicle_age_days'] = 0
    
    # 车辆年龄（年）
    data['vehicle_age_years'] = data['vehicle_age_days'] / 365
    
    # 注册年份和月份
    data['reg_year'] = data['regDate'].dt.year
    data['reg_month'] = data['regDate'].dt.month
    data['reg_day'] = data['regDate'].dt.day
    
    # 创建年份和月份
    data['creat_year'] = data['creatDate'].dt.year
    data['creat_month'] = data['creatDate'].dt.month
    data['creat_day'] = data['creatDate'].dt.day
    
    # 是否为新车（使用年限<1年）
    data['is_new_car'] = (data['vehicle_age_years'] < 1).astype(int)
    
    # 季节特征
    data['reg_season'] = data['reg_month'].apply(lambda x: (x%12 + 3)//3)
    data['creat_season'] = data['creat_month'].apply(lambda x: (x%12 + 3)//3)
    
    # 每年行驶的公里数
    data['km_per_year'] = data['kilometer'] / (data['vehicle_age_years'] + 0.1)
    
    # 车龄分段
    data['age_segment'] = pd.cut(data['vehicle_age_years'], 
                                bins=[-0.01, 1, 3, 5, 10, 100], 
                                labels=['0-1年', '1-3年', '3-5年', '5-10年', '10年以上'])
    
    return data


In [6]:
# 创建时间特征
data = create_time_features(data)
data[['regDate', 'creatDate', 'reg_year', 'reg_month', 'reg_day', 'creat_year', 'creat_month', 'creat_day', 
      'vehicle_age_days', 'vehicle_age_years', 'is_new_car', 'reg_season', 'creat_season', 'km_per_year', 'age_segment']]

创建时间特征...


,regDate,creatDate,reg_year,reg_month,reg_day,creat_year,creat_month,creat_day,vehicle_age_days,vehicle_age_years,is_new_car,reg_season,creat_season,km_per_year,age_segment
0,2004-04-02,2016-04-04,2004,4,2,2016,4,4,4385,12.013699,0,2,2,1.031890,10年以上
1,2003-03-01,2016-03-09,2003,3,1,2016,3,9,4757,13.032877,0,2,2,1.142172,10年以上
2,2004-04-03,2016-04-02,2004,4,3,2016,4,2,4382,12.005479,0,2,2,1.032590,10年以上
3,1996-09-08,2016-03-12,1996,9,8,2016,3,12,7125,19.520548,0,4,2,0.764505,10年以上
4,2012-01-03,2016-03-13,2012,1,3,2016,3,13,1531,4.194521,0,1,2,1.164274,3-5年
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,2004-10-05,2016-03-09,2004,10,5,2016,3,9,4173,11.432877,0,4,2,1.300630,10年以上
199996,2013-04-09,2016-03-23,2013,4,9,2016,3,23,1079,2.956164,0,2,2,1.308830,1-3年
199997,2004-12-11,2016-03-16,2004,12,11,2016,3,16,4113,11.268493,0,1,2,1.099530,10年以上
199998,2002-07-02,2016-03-27,2002,7,2,2016,3,27,5017,13.745205,0,3,2,1.083408,10年以上


In [7]:
def create_car_features(data):
    """
    创建车辆特征
    """
    print("创建车辆特征...")
    
    # 缺失值处理
    numerical_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']
    for feature in numerical_features:
        # 标记缺失值
        data[f'{feature}_missing'] = data[feature].isnull().astype(int)
        # 填充缺失值
        data[feature] = data[feature].fillna(data[feature].median())
    
    # 将model转换为数值型特征
    data['model_num'] = data['model'].astype('category').cat.codes
    #data['model_num'] = data['model'].astype('int') # 不能这么写，因为有一个为空缺值
    
    # 品牌与车型组合
    data['brand_model'] = data['brand'].astype(str) + '_' + data['model'].astype(str)
        
    # 相对年份特征
    current_year = datetime.datetime.now().year
    data['car_age_from_now'] = current_year - data['reg_year']
    
    # 处理异常值
    numerical_cols = ['power', 'kilometer', 'v_0']
    for col in numerical_cols:
        Q1 = data[col].quantile(0.05)
        Q3 = data[col].quantile(0.95)
        IQR = Q3 - Q1
        data[f'{col}_outlier'] = ((data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))).astype(int)
        data[col] = data[col].clip(Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    
    return data

In [8]:
# 创建车辆特征
data = create_car_features(data)
data

创建车辆特征...


,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_11_missing,v_12_missing,v_13_missing,v_14_missing,model_num,brand_model,car_age_from_now,power_outlier,kilometer_outlier,v_0_outlier
0,0,736,2004-04-02,30.0,6,1.0,0.0,0.0,60,12.5,...,0,0,0,0,30,6_30.0,21,0,0,0
1,1,2262,2003-03-01,40.0,1,2.0,0.0,0.0,0,15.0,...,0,0,0,0,40,1_40.0,22,0,0,0
2,2,14874,2004-04-03,115.0,15,1.0,0.0,0.0,163,12.5,...,0,0,0,0,115,15_115.0,21,0,0,0
3,3,71865,1996-09-08,109.0,10,0.0,0.0,1.0,193,15.0,...,0,0,0,0,109,10_109.0,29,0,0,0
4,4,111080,2012-01-03,110.0,5,1.0,0.0,0.0,68,5.0,...,0,0,0,0,110,5_110.0,13,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,249995,111443,2004-10-05,4.0,4,0.0,NaN,1.0,150,15.0,...,0,0,0,0,4,4_4.0,21,0,0,0
199996,249996,152834,2013-04-09,65.0,1,0.0,0.0,0.0,179,4.0,...,0,0,0,0,65,1_65.0,12,0,0,0
199997,249997,132531,2004-12-11,4.0,4,0.0,0.0,1.0,147,12.5,...,0,0,0,0,4,4_4.0,21,0,0,0
199998,249998,143405,2002-07-02,40.0,1,4.0,0.0,1.0,176,15.0,...,0,0,0,0,40,1_40.0,23,0,0,0


In [9]:
def create_statistical_features(data, train_idx):
    """
    创建统计特征
    """
    print("创建统计特征...")
    
    # 仅使用训练集数据创建统计特征
    train_data = data.iloc[train_idx].reset_index(drop=True)
    
    # 品牌级别统计
    brand_stats = train_data.groupby('brand').agg(
        brand_price_mean=('price', 'mean'),
        brand_price_median=('price', 'median'),
        brand_price_std=('price', 'std'),
        brand_price_count=('price', 'count')
    ).reset_index()
    

    # 合并统计特征
    data = data.merge(brand_stats, on='brand', how='left')
    
    # 相对价格特征（相对于平均价格）
    data['brand_price_ratio'] = data['brand_price_mean'] / data['brand_price_mean'].mean()
    
    return data

In [10]:
# 找回训练集的索引
train_idx = data[data['source'] == 'train'].index
test_idx = data[data['source'] == 'test'].index

# 创建统计特征
data = create_statistical_features(data, train_idx)
data

创建统计特征...


,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,brand_model,car_age_from_now,power_outlier,kilometer_outlier,v_0_outlier,brand_price_mean,brand_price_median,brand_price_std,brand_price_count,brand_price_ratio
0,0,736,2004-04-02,30.0,6,1.0,0.0,0.0,60,12.5,...,6_30.0,21,0,0,0,3611.840266,1800.0,4681.293524,10217,0.610000
1,1,2262,2003-03-01,40.0,1,2.0,0.0,0.0,0,15.0,...,1_40.0,22,0,0,0,9273.311947,6499.0,9369.631497,13794,1.566160
2,2,14874,2004-04-03,115.0,15,1.0,0.0,0.0,163,12.5,...,15_115.0,21,0,0,0,9858.582990,8500.0,5425.058140,1458,1.665006
3,3,71865,1996-09-08,109.0,10,0.0,0.0,1.0,193,15.0,...,10_109.0,29,0,0,0,8470.804197,5400.0,8988.307535,14249,1.430626
4,4,111080,2012-01-03,110.0,5,1.0,0.0,0.0,68,5.0,...,5_110.0,13,0,0,0,3306.349411,2300.0,3343.624586,4665,0.558406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,249995,111443,2004-10-05,4.0,4,0.0,NaN,1.0,150,15.0,...,4_4.0,21,0,0,0,8538.788074,6000.0,8472.393191,16737,1.442107
199996,249996,152834,2013-04-09,65.0,1,0.0,0.0,0.0,179,4.0,...,1_65.0,12,0,0,0,9273.311947,6499.0,9369.631497,13794,1.566160
199997,249997,132531,2004-12-11,4.0,4,0.0,0.0,1.0,147,12.5,...,4_4.0,21,0,0,0,8538.788074,6000.0,8472.393191,16737,1.442107
199998,249998,143405,2002-07-02,40.0,1,4.0,0.0,1.0,176,15.0,...,1_40.0,23,0,0,0,9273.311947,6499.0,9369.631497,13794,1.566160


In [11]:
def encode_categorical_features(data):
    """
    编码分类特征
    """
    print("编码分类特征...")
    
    # 目标编码的替代方案 - 频率编码
    categorical_cols = ['model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage']
    
    for col in categorical_cols:
        # 填充缺失值
        data[col] = data[col].fillna('未知')
        
        # 频率编码
        freq_encoding = data.groupby(col).size() / len(data)
        data[f'{col}_freq'] = data[col].map(freq_encoding)
    
    # 将分类变量转换为CatBoost可以识别的格式
    for col in categorical_cols:
        data[col] = data[col].astype('str')
    
    return data, categorical_cols

In [12]:
data, categorical_cols = encode_categorical_features(data)
data

编码分类特征...


,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,brand_price_median,brand_price_std,brand_price_count,brand_price_ratio,model_freq,brand_freq,bodyType_freq,fuelType_freq,gearbox_freq,notRepairedDamage_freq
0,0,736,2004-04-02,30.0,6,1.0,0.0,0.0,60,12.5,...,1800.0,4681.293524,10217,0.610000,0.015675,0.068585,0.236160,0.610725,0.743770,0.742925
1,1,2262,2003-03-01,40.0,1,2.0,0.0,0.0,0,15.0,...,6499.0,9369.631497,13794,1.566160,0.030130,0.092105,0.201050,0.610725,0.743770,0.161965
2,2,14874,2004-04-03,115.0,15,1.0,0.0,0.0,163,12.5,...,8500.0,5425.058140,1458,1.665006,0.006210,0.009670,0.236160,0.610725,0.743770,0.742925
3,3,71865,1996-09-08,109.0,10,0.0,0.0,1.0,193,15.0,...,5400.0,8988.307535,14249,1.430626,0.002540,0.094810,0.275925,0.610725,0.216485,0.742925
4,4,111080,2012-01-03,110.0,5,1.0,0.0,0.0,68,5.0,...,2300.0,3343.624586,4665,0.558406,0.003610,0.030750,0.236160,0.610725,0.743770,0.742925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,249995,111443,2004-10-05,4.0,4,0.0,未知,1.0,150,15.0,...,6000.0,8472.393191,16737,1.442107,0.056175,0.111345,0.275925,0.058020,0.216485,0.161965
199996,249996,152834,2013-04-09,65.0,1,0.0,0.0,0.0,179,4.0,...,6499.0,9369.631497,13794,1.566160,0.018240,0.092105,0.275925,0.610725,0.743770,0.742925
199997,249997,132531,2004-12-11,4.0,4,0.0,0.0,1.0,147,12.5,...,6000.0,8472.393191,16737,1.442107,0.056175,0.111345,0.275925,0.610725,0.216485,0.742925
199998,249998,143405,2002-07-02,40.0,1,4.0,0.0,1.0,176,15.0,...,6499.0,9369.631497,13794,1.566160,0.030130,0.092105,0.064335,0.610725,0.216485,0.742925


In [13]:
def feature_selection(data, categorical_cols):
    """
    特征选择和最终数据准备
    """
    print("特征选择和最终数据准备...")
    
    # 删除不再需要的列, 所有车offerType=0,seller只有1个为1，其他都为0
    drop_cols = ['regDate', 'creatDate', 'price', 'SaleID', 'name', 'offerType', 'seller', 'source']
    data = data.drop(drop_cols, axis=1, errors='ignore')
    
    # 确保所有分类特征都被正确标记
    # 添加age_segment到分类特征列表中
    if 'age_segment' not in categorical_cols and 'age_segment' in data.columns:
        categorical_cols.append('age_segment')
    
    # 确保brand_model也被标记为分类特征
    if 'brand_model' not in categorical_cols and 'brand_model' in data.columns:
        categorical_cols.append('brand_model')
    
    # 转换分类特征
    for col in categorical_cols:
        if col in data.columns:
            data[col] = data[col].astype('category')
    
    return data, categorical_cols

In [14]:
data, cat_features = feature_selection(data, categorical_cols)

特征选择和最终数据准备...


In [15]:
X_train_full = data.iloc[train_idx].reset_index(drop=True)
X_test = data.iloc[test_idx].reset_index(drop=True)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y, test_size=0.1, random_state=42
)

In [18]:
joblib.dump(X_train, '../data/processed_data/fe_X_train.joblib')
joblib.dump(X_val, '../data/processed_data/fe_X_val.joblib')
joblib.dump(y_train, '../data/processed_data/fe_y_train.joblib')
joblib.dump(y_val, '../data/processed_data/fe_y_val.joblib')
joblib.dump(X_test, '../data/processed_data/fe_test_data.joblib')
joblib.dump(test_ids, '../data/processed_data/fe_sale_ids.joblib')
joblib.dump(cat_features, '../data/processed_data/fe_cat_features.joblib')

['../data/processed_data/fe_cat_features.joblib']

In [20]:
def train_catboost_model(X_train, X_val, y_train, y_val, cat_features):
    """
    训练CatBoost模型
    """
    print("\n开始训练CatBoost模型...")
    
    # 创建数据池
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)
    
    # 设置模型参数
    params = {
        'iterations': 3000,
        'learning_rate': 0.03,
        'depth': 6,
        'l2_leaf_reg': 3,
        'bootstrap_type': 'Bayesian',
        'random_seed': 42,
        'od_type': 'Iter',
        'od_wait': 100,
        'verbose': 100,
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'task_type': 'CPU',
        'thread_count': -1
    }
    
    # 创建模型
    model = CatBoostRegressor(**params)
    
    # 训练模型
    model.fit(
        train_pool,
        eval_set=val_pool,
        use_best_model=True,
        plot=True
    )
    
    # 保存模型
    model.save_model('../data/processed_data/fe_catboost_model.cbm')
    print("模型已保存到 ../data/processed_data/fe_catboost_model.cbm")
    
    return model

In [22]:
model = train_catboost_model(X_train, X_val, y_train, y_val, cat_features)


开始训练CatBoost模型...


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4376.9572075	test: 4342.5891693	best: 4342.5891693 (0)	total: 863ms	remaining: 43m 8s
100:	learn: 1182.4140324	test: 1163.9740707	best: 1163.9740707 (100)	total: 1m 2s	remaining: 29m 56s
200:	learn: 891.0472403	test: 886.1544448	best: 886.1544448 (200)	total: 1m 42s	remaining: 23m 46s
300:	learn: 807.3433635	test: 811.2896233	best: 811.2896233 (300)	total: 2m 22s	remaining: 21m 20s
400:	learn: 764.7679909	test: 771.9730815	best: 771.9730815 (400)	total: 3m 4s	remaining: 19m 53s
500:	learn: 730.2469913	test: 739.6059471	best: 739.6059471 (500)	total: 3m 45s	remaining: 18m 45s
600:	learn: 703.4456767	test: 714.5427333	best: 714.5427333 (600)	total: 4m 23s	remaining: 17m 33s
700:	learn: 685.5680812	test: 698.0527852	best: 698.0527852 (700)	total: 5m 2s	remaining: 16m 30s
800:	learn: 667.3295994	test: 681.3670461	best: 681.3670461 (800)	total: 5m 41s	remaining: 15m 37s
900:	learn: 652.3245145	test: 667.7154871	best: 667.7154871 (900)	total: 6m 14s	remaining: 14m 33s
1000:	learn: 

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
model = CatBoostRegressor()
model.load_model('../data/processed_data/fe_catboost_model.cbm')

# 加载测试集数据和SaleID
X_test = joblib.load('../data/processed_data/fe_test_data.joblib')
test_ids = joblib.load('../data/processed_data/fe_sale_ids.joblib')

X_val = joblib.load('../data/processed_data/fe_X_val.joblib')
y_val = joblib.load('../data/processed_data/fe_y_val.joblib')

In [12]:
cat_features = [
    'model',
    'brand',
    'bodyType',
    'fuelType',
    'gearbox',
    'notRepairedDamage',
    'age_segment',
    'brand_model'
]

In [15]:
import matplotlib.pyplot as plt
def evaluate_model(model, X_val, y_val, cat_features):
    """
    评估模型性能
    """
    # 创建验证数据池
    val_pool = Pool(X_val, cat_features=cat_features)
    
    # 预测
    y_pred = model.predict(val_pool)
    
    # 计算评估指标
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    print("\n模型评估结果：")
    print(f"均方根误差 (RMSE): {rmse:.2f}")
    print(f"平均绝对误差 (MAE): {mae:.2f}")
    print(f"R2分数: {r2:.4f}")
    
    # 绘制预测值与实际值的对比图
    plt.figure(figsize=(10, 6))
    plt.scatter(y_val, y_pred, alpha=0.5)
    plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
    plt.xlabel('实际价格')
    plt.ylabel('预测价格')
    plt.title('CatBoost预测价格 vs 实际价格')
    plt.tight_layout()
    plt.savefig('fe_catboost_prediction_vs_actual.png')
    plt.close()
    
    return rmse, mae, r2

In [21]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
rmse, mae, r2 = evaluate_model(model, X_val, y_val, cat_features)

findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei



模型评估结果：
均方根误差 (RMSE): 1355.01
平均绝对误差 (MAE): 557.91
R2分数: 0.9661


findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not foun

In [17]:
def plot_feature_importance(model, X_train):
    """
    绘制特征重要性图
    """
    # 获取特征重要性
    feature_importance = model.get_feature_importance()
    feature_names = X_train.columns
    
    # 创建特征重要性DataFrame
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    })
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    # 保存特征重要性到CSV
    importance_df.to_csv('fe_catboost_feature_importance.csv', index=False)
    
    # 绘制特征重要性图
    plt.figure(figsize=(14, 8))
    sns.barplot(x='importance', y='feature', data=importance_df.head(20))
    plt.title('CatBoost Top 20 特征重要性')
    plt.tight_layout()
    plt.savefig('fe_catboost_feature_importance.png')
    plt.close()
    
    return importance_df

In [22]:
X_train = joblib.load('../data/processed_data/fe_X_train.joblib')
importance_df = plot_feature_importance(model, X_train)

findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not foun

In [23]:
def predict_test_data(model, X_test, test_ids, cat_features):
    """
    预测测试集数据
    """
    print("\n正在预测测试集...")
    
    # 创建测试数据池
    test_pool = Pool(X_test, cat_features=cat_features)
    
    # 预测
    predictions = model.predict(test_pool)
    
    # 创建提交文件
    submit_data = pd.DataFrame({
        'SaleID': test_ids,
        'price': predictions
    })
    
    # 保存预测结果
    submit_data.to_csv('fe_catboost_submit_result.csv', index=False)
    print("预测结果已保存到 fe_catboost_submit_result.csv")

# 预测测试集
predict_test_data(model, X_test, test_ids, cat_features)

print("\n模型训练、评估和预测完成！")
print(f"Top 10 重要特征:\n{importance_df.head(10)}")


正在预测测试集...
预测结果已保存到 fe_catboost_submit_result.csv

模型训练、评估和预测完成！
Top 10 重要特征:
        feature  importance
12          v_3   25.467024
21         v_12   18.092635
9           v_0   17.668084
5         power    5.843721
17          v_8    3.912581
6     kilometer    3.507068
18          v_9    3.158150
19         v_10    2.953284
15          v_6    2.513120
36  age_segment    2.091691
